# Get Fundamentals Data. 

In [1]:
# Python module. 
import re, os  
import pandas as pd

# Change the current directory from (./notebook) to root directory. 
while not re.match(r".+MADS-CAP$", os.getcwd()): 
	os.chdir("..") 
	
print(f"Current directory: ({os.getcwd()})") 

# For clearing safe warnings. Not important. 
from IPython.display import clear_output

# Custom modules. 
from source.modules.processor_features import (
	compile_features_each_ticker, rolling_sum_bygroup, 
)
from source.modules.processor_fundamental import (
	get_snp_wikiinfo, get_corporate_overview_alphav, get_fundamental_alphav, 
	compute_extra_fundamental, score_fundamental, rank_fundamental, 
	within_sector_diff, rank_within_sector_diff, 
) 

# Custom configs. 
from source.config_py.config import (
	DIR_DATASET_FUNDAMENTAL, DIR_DATASET_UTIL, 
	TICKER_FROM_SNP, TICKER_TO_COLLECT, TICKER_TO_EXCLUDE, 
	FINANCIAL_INCOME_STATES, FINANCIAL_EVAL_QUALITY, 
)

Current directory: (/Users/lioneltay/Dropbox/Courses/michigan_mads/SIADS_697_/submission/MADS-CAP)


## Configurations (general). 

In [2]:
# Pandas DF config. 
pd.set_option("display.max_rows", 50, "display.max_columns", 100, "display.max_colwidth", 50)

# List of ticker to collect data. 
snp = pd.read_csv(f"{DIR_DATASET_UTIL}/snp500_tickers.csv").Symbol if TICKER_FROM_SNP else set() 
ticker_to_collect = TICKER_TO_COLLECT.union(snp).difference(TICKER_TO_EXCLUDE) 

# Whether to load the cache file for the fundamental data. 
load_cache = True 

# For clearing the output. Not important. 
clear_output() 

## Get sector info. 

In [3]:
df_snp_info = get_snp_wikiinfo() 

# Preview. 
df_snp_info 

Getting S&P sector info from (Wikipedia).


,ticker,security,sec_filings,gics_sector,gics_sub_industry,headquarters_location,date_first_added,cik,founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


### Cache dataset. 

In [4]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_UTIL, "snp500_sector_info.csv") 
# df_snp_info.to_csv(filepath, index=False) 

## Get corporate info. 

In [5]:
# # Assign (load_cache) to (False) to overwrite existing data for all tickers. 
# # Otherwise, it will append unavailable data to the existing dataset. 
# # Take about 15 minutes to complete the entire S&P tickers. 

# filepath = os.path.join(DIR_DATASET_UTIL, "corporate_overview.csv") 
# df_corporate = compile_features_each_ticker(get_corporate_overview_alphav, filepath, ticker_to_collect, load_cache=load_cache) 

# # Preview. 
# df_corporate 

## Get fundamental data. 

In [6]:
# Assign (load_cache) to (False) to overwrite existing data for all tickers. 
# Otherwise, it will append unavailable data to the existing dataset. 
# Take about 30 minutes to complete the entire S&P tickers. 

filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_annual.csv") 
df_finstate_ann = compile_features_each_ticker(
	get_fundamental_alphav, filepath, ticker_to_collect, load_cache=load_cache, **{"annual": True}
) 

# Preview. 
df_finstate_ann

,ticker,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_y
0,CINF,2017-12-31,USD,-2.009000e+09,5.732000e+09,7.741000e+09,4.857000e+09,-2.022000e+09,NaN,NaN,1.300000e+07,605000000.0,-53000000.0,-53000000.0,53000000.0,4.970000e+09,NaN,28000000.0,55000000.0,7.300000e+08,-3.150000e+08,53000000.0,1.045000e+09,1.648000e+09,7.830000e+08,8.380000e+08,1.045000e+09,2.184300e+10,6.570000e+08,6.570000e+08,6.570000e+08,NaN,NaN,1.850000e+08,1.850000e+08,2.530000e+08,6.700000e+08,670000000.0,NaN,1.705100e+10,1.724600e+10,5.190000e+08,-2.216000e+09,-1.469000e+10,1.360000e+10,1.281300e+10,NaN,NaN,NaN,2.400000e+07,8.270000e+08,4.000000e+07,7.870000e+08,24000000.0,NaN,7.870000e+08,1.278900e+10,1.200400e+10,8.243000e+09,1.387000e+09,5.180000e+09,397000000.0,198300000.0,1.052000e+09,2.962000e+09,NaN,2.400000e+07,-101000000.0,9.410000e+08,1.600000e+07,NaN,NaN,1.045000e+09,-5.580000e+08,-614000000.0,50000000.0,92000000.0,92000000.0,NaN,400000000.0,400000000.0,NaN,NaN,NaN,NaN,-79000000.0,NaN,-120000000.0,NaN,1.045000e+09
1,CINF,2018-12-31,USD,-2.566000e+09,5.407000e+09,7.973000e+09,5.012000e+09,-2.582000e+09,NaN,NaN,1.600000e+07,614000000.0,-53000000.0,-53000000.0,53000000.0,5.185000e+09,NaN,31000000.0,63000000.0,2.510000e+08,-3.600000e+07,53000000.0,2.870000e+08,2.400000e+07,3.040000e+08,3.670000e+08,2.870000e+08,2.193500e+10,7.840000e+08,7.840000e+08,7.840000e+08,NaN,NaN,1.950000e+08,1.950000e+08,2.650000e+08,7.380000e+08,738000000.0,NaN,1.673200e+10,1.693500e+10,NaN,-2.331000e+09,-1.415200e+10,1.410200e+10,1.331400e+10,NaN,NaN,NaN,3.200000e+07,8.340000e+08,4.600000e+07,7.880000e+08,32000000.0,NaN,7.880000e+08,1.328200e+10,1.260900e+10,7.833000e+09,1.492000e+09,7.625000e+09,397000000.0,198300000.0,1.181000e+09,3.005000e+09,NaN,0.000000e+00,199000000.0,9.770000e+08,2.000000e+07,2000000.0,NaN,2.870000e+08,-4.510000e+08,-603000000.0,0.0,125000000.0,125000000.0,NaN,336000000.0,336000000.0,NaN,NaN,NaN,NaN,-116000000.0,NaN,127000000.0,NaN,2.870000e+08
2,CINF,2019-12-31,USD,-4.430000e+08,7.924000e+09,8.367000e+09,5.290000e+09,-4.790000e+08,4.000000e+06,NaN,3.

In [7]:
# Assign (load_cache) to (False) to overwrite existing data for all tickers. 
# Otherwise, it will append unavailable data to the existing dataset. 
# Take about 30 minutes to complete the entire S&P tickers. 

filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_quarter.csv") 
df_finstate_qrt = compile_features_each_ticker(
	get_fundamental_alphav, filepath, ticker_to_collect, load_cache=load_cache, **{"annual": False}
) 

# Preview. 
df_finstate_qrt 

,ticker,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_y
0,CINF,2017-06-30,USD,-5.510000e+08,1.386000e+09,1.937000e+09,1.232000e+09,-5.550000e+08,NaN,NaN,4.000000e+06,150000000.0,-13000000.0,NaN,13000000.0,1.245000e+09,NaN,NaN,13000000.0,128000000.0,28000000.0,13000000.0,100000000.0,245000000.0,141000000.0,1.540000e+08,100000000.0,2.123800e+10,6.060000e+08,6.060000e+08,6.060000e+08,NaN,NaN,1.840000e+08,1.840000e+08,245000000.0,NaN,NaN,NaN,1.639400e+10,1.657800e+10,7.140000e+08,662000000.0,-1.399100e+10,1.386500e+10,1.307200e+10,NaN,NaN,NaN,1.700000e+07,8.260000e+08,NaN,7.870000e+08,843000000.0,NaN,7.870000e+08,2.461000e+09,1.200100e+10,7.373000e+09,1.372000e+09,5.174000e+09,397000000.0,198300000.0,309000000.0,695000000.0,NaN,57000000.0,79000000.0,229000000.0,7000000.0,-7000000.0,NaN,100000000.0,-8.700000e+07,-1.590000e+08,0.0,5.500000e+07,5.500000e+07,NaN,81000000.0,81000000.0,NaN,NaN,NaN,NaN,-5.300000e+07,NaN,6.300000e+07,NaN,100000000.0
1,CINF,2017-09-30,USD,-5.730000e+08,1.412000e+09,1.985000e+09,1.257000e+09,-5.760000e+08,NaN,NaN,3.000000e+06,152000000.0,-13000000.0,NaN,13000000.0,1.250000e+09,NaN,NaN,13000000.0,129000000.0,27000000.0,13000000.0,102000000.0,224000000.0,142000000.0,1.550000e+08,102000000.0,2.159200e+10,6.740000e+08,6.740000e+08,6.740000e+08,NaN,NaN,1.860000e+08,1.860000e+08,251000000.0,NaN,NaN,NaN,1.666400e+10,1.684800e+10,6.860000e+08,650000000.0,-1.423600e+10,1.406900e+10,1.327600e+10,NaN,NaN,NaN,1.700000e+07,8.260000e+08,NaN,7.870000e+08,843000000.0,NaN,7.870000e+08,2.475000e+09,1.213900e+10,7.523000e+09,1.370000e+09,5.193000e+09,397000000.0,198300000.0,301000000.0,725000000.0,NaN,36000000.0,-35000000.0,237000000.0,5000000.0,2000000.0,NaN,102000000.0,-1.260000e+08,-1.070000e+08,0.0,0.000000e+00,0.000000e+00,NaN,81000000.0,81000000.0,NaN,NaN,NaN,NaN,2.000000e+06,NaN,6.800000e+07,NaN,102000000.0
2,CINF,2017-12-31,USD,1.411000e+09,1.411000e+09,1.899000e+09,1.185000e+09,1.411000e+09,NaN,NaN,2.000000e+06,155000000.0,-14000000.0,NaN,14000000.0,1.262000e+09,NaN,NaN,15000000.0,197000000.0,-445000000.0,14000000.0,1

## Compute additional fundamental data. 

In [8]:
# Compute change / growth rate, arithmetic, and ratio to create additional fundamental data.
df_finstate_ann_ext = compute_extra_fundamental(df_finstate_ann) 

# Preview. 
df_finstate_ann_ext 

Compute additional fundamental data.


,ticker,fiscalDateEnding,totalRevenue,researchAndDevelopment,capitalExpenditures,depreciationAndAmortization,grossProfit,interestExpense,ebit,incomeBeforeTax,netIncomeFromContinuingOperations,netIncome_x,inventory,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiabilities,longTermDebtNoncurrent,totalShareholderEquity,commonStockSharesOutstanding,operatingCashflow,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,ownerEarnings,assetsQuick,workingCapital,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,eps,eps_continuing,rps
0,CINF,2017-12-31,5.732000e+09,NaN,1.600000e+07,55000000.0,-2.009000e+09,53000000.0,7.830000e+08,7.300000e+08,1.045000e+09,1.045000e+09,NaN,2.184300e+10,6.570000e+08,1.281300e+10,1.360000e+10,NaN,8.243000e+09,198300000.0,1.052000e+09,NaN,NaN,NaN,NaN,NaN,NaN,1.084000e+09,NaN,-1.215600e+10,0.262418,0.183531,-0.350488,0.182310,5.269793,5.269793,28.905698
1,CINF,2018-12-31,5.407000e+09,NaN,2.000000e+07,63000000.0,-2.566000e+09,53000000.0,3.040000e+08,2.510000e+08,2.870000e+08,2.870000e+08,NaN,2.193500e+10,7.840000e+08,1.331400e+10,1.410200e+10,NaN,7.833000e+09,198300000.0,1.181000e+09,-0.056699,0.277252,-0.725359,-0.611750,-0.725359,0.122624,3.300000e+08,NaN,-1.253000e+10,0.246501,0.218421,-0.474570,0.053079,1.447302,1.447302,27.266768
2,CINF,2019-12-31,7.924000e+09,NaN,2.400000e+07,9000000.0,-4.430000e+08,53000000.0,2.525000e+09,2.472000e+09,1.997000e+09,1.997000e+09,NaN,2.540800e+10,7.670000e+08,1.475600e+10,1.554400e+10,NaN,9.864000e+09,198300000.0,1.208000e+09,0.465508,-0.827358,5.958188,7.305921,5.958188,0.022862,1.982000e+09,NaN,-1.398900e+10,0.311870,0.152448,-0.055906,0.252019,10.070600,10.070600,39.959657
3,CINF,2020-12-31,7.536000e+09,NaN,2.000000e+07,15000000.0,-1.900000e+09,54000000.0,1.553000e+09,1.499000e+09,1.216000e+09,1.216000e+09,NaN,2.754200e+10,9.000000e+08,5.400000e+07,1.675300e+10,NaN,1.078900e+10,160900000.0,1.491000e+09,-0.048965,3.288939,-0.391087,-0.384950,-0.391087,0.234272,1.211000e+09,NaN,8.460000e+08,0.273618,0.197850,-0.252123,0.161359,7.557489,7.557489,46.836544
4,CINF,2021-12-31,9.630000e+09,NaN,1.500000e+07,15000000.0,4.970000e+08,53000000.0,3.723000e+09,3.670000e+09,2.946000e+09,2.946000e+09,NaN,3.138700e+10,1.139000e+09,5.400000e+07,1.828200e+10,NaN,1.310500e+10,160700000.0,1.981000e+09,0.277866,-1.261579,1.422697,1.397296,1.422697,0.328638,2.946000e+09,NaN,1.085000e+09,0.306815,0.205711,0.051610,0.305919,18.332296,18.332296,59.925327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477,TWTR,2017-12-31,2.443299e+09,5.420100e+08,1.607420e+08,395867000.0,1.582057e+09,105237000.0,9.819000e+06,-9.541800e+07,-1.080630e+08,-1.080630e+08,NaN,7.412477e+09,5.321884e+09,5.832780e+08,2.365259e+09,1.627460e+09,5.047218e+09,746902000.0,8.312090e+08,NaN,NaN,NaN,NaN,NaN,NaN,1.270620e+08,NaN,4.738606e+09,0.329620,0.340199,0.647509,-0.044228,-0.144682,-0.144682,3.271244
2478,TWTR,2018-12-31,3.042359e+09,5.538580e+08,4.839340e+08,425498000.0,2.077362e+09,132606000.0,5.561500e+08,4.235440e+08,1.205596e+09,1.205596e+09,NaN,1.016257e+10,7.111036e+09,1.516311e+09,3.356978e+09,1.730922e+09,6.805594e+09,764257000.0,1.339711e+09,0.245185,0.313077,-12.156418,55.640187,-12.156418,0.611762,1.147160e+09,NaN,5.594725e+09,0.299369,0.440353,0.682813,0.396270,1.577475,1.577475,3.980806
2479,TWTR,2019-12-31,3.459329e+09,6.822810e+08,5.406880e+08,465549000.0,2.322288e+09,138180000.0,5.283190e+08,3.901390e+08,1.465659e+09,1.465659e+09,NaN,1.270339e+10,7.620075e+09,8.324760e+08,3.999003e+09,2.508800e+09,8.704386e+09,764257000.0,1.303364e+09,0.137055,0.117902,0.215713,-0.050042,0.215713,-0.027130,1.390520e+09,NaN,6.787599e+09,0.272315,0.376768,0.671312,0.423683,1.917757,1.917757,4.526395
2480,TWTR,2020-12-31,3.716349e+09,8.730110e+08,8.733540e+08,495177000.0,2.349961e+09,152878000.0

In [9]:
# Compute change / growth rate, arithmetic, and ratio to create additional fundamental data.
df_finstate_qrt_ext = compute_extra_fundamental(df_finstate_qrt) 

# Preview. 
df_finstate_qrt_ext 

Compute additional fundamental data.


,ticker,fiscalDateEnding,totalRevenue,researchAndDevelopment,capitalExpenditures,depreciationAndAmortization,grossProfit,interestExpense,ebit,incomeBeforeTax,netIncomeFromContinuingOperations,netIncome_x,inventory,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiabilities,longTermDebtNoncurrent,totalShareholderEquity,commonStockSharesOutstanding,operatingCashflow,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,ownerEarnings,assetsQuick,workingCapital,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,eps,eps_continuing,rps
0,CINF,2017-06-30,1.386000e+09,NaN,7000000.0,13000000.0,-5.510000e+08,13000000.0,141000000.0,128000000.0,100000000.0,100000000.0,NaN,2.123800e+10,6.060000e+08,1.307200e+10,1.386500e+10,NaN,7.373000e+09,198300000.0,309000000.0,NaN,NaN,NaN,NaN,NaN,NaN,106000000.0,NaN,-1.246600e+10,0.065260,0.222944,-0.397547,0.072150,0.504286,0.504286,6.989410
1,CINF,2017-09-30,1.412000e+09,NaN,5000000.0,13000000.0,-5.730000e+08,13000000.0,142000000.0,129000000.0,102000000.0,102000000.0,NaN,2.159200e+10,6.740000e+08,1.327600e+10,1.406900e+10,NaN,7.523000e+09,198300000.0,301000000.0,0.018759,0.039927,0.020000,0.007092,0.020000,-0.025890,110000000.0,NaN,-1.260200e+10,0.065395,0.213173,-0.405807,0.072238,0.514372,0.514372,7.120524
2,CINF,2017-12-31,1.411000e+09,NaN,2000000.0,15000000.0,1.411000e+09,14000000.0,211000000.0,197000000.0,197000000.0,642000000.0,NaN,2.184300e+10,6.570000e+08,1.281300e+10,1.360000e+10,NaN,8.243000e+09,198300000.0,306000000.0,-0.000708,-3.462478,0.931373,0.485915,5.294118,0.016611,655000000.0,NaN,-1.215600e+10,0.064597,0.216867,1.000000,0.139617,3.237519,0.993444,7.115482
3,CINF,2018-03-31,1.224000e+09,NaN,3000000.0,18000000.0,-7.180000e+08,13000000.0,-37000000.0,-50000000.0,-31000000.0,-31000000.0,NaN,2.147000e+10,6.040000e+08,1.273700e+10,1.352400e+10,NaN,7.946000e+09,198300000.0,154000000.0,-0.132530,-1.508859,-1.157360,-1.175355,-1.048287,-0.496732,-16000000.0,NaN,-1.213300e+10,0.057010,0.125817,-0.586601,-0.025327,-0.156329,-0.156329,6.172466
4,CINF,2018-06-30,1.558000e+09,NaN,6000000.0,14000000.0,-4.440000e+08,13000000.0,277000000.0,264000000.0,217000000.0,217000000.0,NaN,2.176000e+10,5.210000e+08,1.305700e+10,1.384400e+10,NaN,7.916000e+09,198300000.0,310000000.0,0.272876,-0.381616,-8.000000,-8.486486,-8.000000,1.012987,225000000.0,NaN,-1.253600e+10,0.071599,0.198973,-0.284981,0.139281,1.094302,1.094302,7.856783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9969,TWTR,2021-03-31,1.036018e+09,250709000.0,181181000.0,131052000.0,6.550100e+08,13185000.0,63189000.0,50004000.0,68004000.0,68005000.0,NaN,1.498408e+10,9.863742e+09,2.036693e+09,7.247425e+09,4.246025e+09,7.736650e+09,798083000.0,390184000.0,-0.196288,-0.234906,-0.693836,-0.760815,-0.693831,0.181302,17876000.0,NaN,7.827049e+09,0.069141,0.376619,0.632238,0.065640,0.085210,0.085209,1.298133
9970,TWTR,2021-06-30,1.190427e+09,299859000.0,279195000.0,134812000.0,7.734950e+08,13893000.0,95193000.0,81300000.0,65649000.0,65649000.0,NaN,1.532322e+10,9.728141e+09,2.257919e+09,7.607097e+09,4.248338e+09,7.716128e+09,797941000.0,381967000.0,0.149041,0.180890,-0.034630,0.506481,-0.034645,-0.021059,-78734000.0,NaN,7.470222e+09,0.077688,0.320866,0.649763,0.055147,0.082273,0.082273,1.491873
9971,TWTR,2021-09-30,1.283817e+09,324252000.0,411227000.0,133448000.0,7.993380e+08,13284000.0,-713773000.0,-727057000.0,-536782000.0,-536757000.0,NaN,1.458934e+10,8.649234e+09,2.114981e+09,7.416714e+09,4.250678e+09,7.172631e+09,799612000.0,388980000.0,0.078451,0.033411,-9.176545,-8.498167,-9.176164,0.018360,-814536000.0,NaN,6.534253e+09,0.087997,0.302987,0.622626,-0.418114,-0.671272,-0.671303,1.605550
9972,TWTR,2021-12-31,1.567220e+09,371884000.0,139943000.0,145536000.0,1.052129e+09,10824000.0,195514000.0,184690000.0,181669000.0,181694000.0

In [10]:
# Calculate the rolling sum for quarterly report. 
df_finstate_qrt_roll = rolling_sum_bygroup(
	df_finstate_qrt, groupby=["ticker"], window=4, usecols=FINANCIAL_INCOME_STATES
) 

# Compute change / growth rate, arithmetic, and ratio to create additional fundamental data.
df_finstate_qrt_roll_ext = compute_extra_fundamental(df_finstate_qrt_roll) 

# Preview. 
df_finstate_qrt_roll_ext 

Compute additional fundamental data.


,ticker,fiscalDateEnding,totalRevenue,researchAndDevelopment,capitalExpenditures,depreciationAndAmortization,grossProfit,interestExpense,ebit,incomeBeforeTax,netIncomeFromContinuingOperations,netIncome_x,inventory,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiabilities,longTermDebtNoncurrent,totalShareholderEquity,commonStockSharesOutstanding,operatingCashflow,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,ownerEarnings,assetsQuick,workingCapital,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,eps,eps_continuing,rps
0,CINF,2017-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.123800e+10,6.060000e+08,1.307200e+10,1.386500e+10,NaN,7.373000e+09,198300000.0,309000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.246600e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CINF,2017-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.159200e+10,6.740000e+08,1.327600e+10,1.406900e+10,NaN,7.523000e+09,198300000.0,301000000.0,NaN,NaN,NaN,NaN,NaN,-0.025890,NaN,NaN,-1.260200e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CINF,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.184300e+10,6.570000e+08,1.281300e+10,1.360000e+10,NaN,8.243000e+09,198300000.0,306000000.0,NaN,NaN,NaN,NaN,NaN,0.016611,NaN,NaN,-1.215600e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CINF,2018-03-31,5.433000e+09,NaN,1.700000e+07,59000000.0,-4.310000e+08,53000000.0,457000000.0,404000000.0,3.680000e+08,8.130000e+08,NaN,2.147000e+10,6.040000e+08,1.273700e+10,1.352400e+10,NaN,7.946000e+09,198300000.0,154000000.0,NaN,NaN,NaN,NaN,NaN,-0.496732,8.550000e+08,NaN,-1.213300e+10,0.253051,0.028345,-0.079330,0.067734,4.099849,1.855774,27.397882
4,CINF,2018-06-30,5.605000e+09,NaN,1.600000e+07,60000000.0,-3.240000e+08,53000000.0,593000000.0,540000000.0,4.850000e+08,9.300000e+08,NaN,2.176000e+10,5.210000e+08,1.305700e+10,1.384400e+10,NaN,7.916000e+09,198300000.0,310000000.0,0.031658,-0.248260,0.317935,0.297593,0.143911,1.012987,9.740000e+08,NaN,-1.253600e+10,0.257583,0.055308,-0.057806,0.086530,4.689864,2.445789,28.265255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9969,TWTR,2021-03-31,3.944730e+09,9.233320e+08,9.318680e+08,505580000.0,2.481371e+09,132793000.0,147369000.0,14576000.0,-1.059202e+09,-1.059225e+09,NaN,1.498408e+10,9.863742e+09,2.036693e+09,7.247425e+09,4.246025e+09,7.736650e+09,798083000.0,390184000.0,0.061453,0.055920,-0.067277,0.445999,-0.067277,0.181302,-1.485513e+09,NaN,7.827049e+09,0.263261,0.098913,0.629034,-0.268511,-1.327212,-1.327183,4.942757
9970,TWTR,2021-06-30,4.451719e+09,1.007385e+09,1.046647e+09,516555000.0,2.859467e+09,106858000.0,491840000.0,384982000.0,3.844520e+08,3.844290e+08,NaN,1.532322e+10,9.728141e+09,2.257919e+09,7.607097e+09,4.248338e+09,7.716128e+09,797941000.0,381967000.0,0.128523,0.152374,-1.362964,2.337473,-1.362934,-0.021059,-1.456630e+08,NaN,7.470222e+09,0.290521,0.085802,0.642329,0.086360,0.481776,0.481805,5.579008
9971,TWTR,2021-09-30,4.799303e+09,1.122760e+09,1.167128e+09,528020000.0,3.083960e+09,80528000.0,-291206000.0,-371734000.0,-1.810130e+08,-1.809870e+08,NaN,1.458934e+10,8.649234e+09,2.114981e+09,7.416714e+09,4.250678e+09,7.172631e+09,799612000.0,388980000.0,0.078079,0.078509,-1.470834,-1.592075,-1.470794,0.018360,-8.200950e+08,NaN,6.534253e+09,0.328959,0.081049,0.642585,-0.037717,-0.226344,-0.226376,6.002040
9972,TWTR,2021-12-31,5.077482e+09,1.246704e+09,1.011546e+09,544848000.0,3.279972e+09,51186000.0,-359877000.0,-411063000.0,-2.214600e+08,-2.214090e+08,NaN,1.405952e+10,7.918370e+09,1.343867e+09,6.752317e+09,4.253019e+09,7.307199e+09,799384000.0,-528442000.0,0.057962,0.063559,0.223448,0.235816,0.223342,-2.358533,-6.881070e+08,NaN,6.574503e+09,0.361142,-0.104076,0.645984,-0.043616,-0.276975,-0.277038,6.351743


### Cache dataset. 

In [11]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_ann_ext.csv") 
# df_finstate_ann_ext.to_csv(filepath, index=False) 

# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_qrt_ext.csv") 
# df_finstate_qrt_ext.to_csv(filepath, index=False) 

# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_qrt_roll_ext.csv") 
# df_finstate_qrt_roll_ext.to_csv(filepath, index=False) 

## Rank corporate fundamental. 

### Rank by score. 

In [12]:
# Rank the fundamental quality. 
df_rank_fundamental = rank_fundamental(score_fundamental, df_finstate_ann_ext) 

# Preview 
df_rank_fundamental

Ranking the fundamental quality.


,ticker,score_growth_totalRevenue,score_growth_grossProfit,score_growth_netIncomeFromOperations,score_growth_ebit,score_growth_netIncome_x,score_growth_operatingCashflow,score_turnover_asset,score_turnover_operatingCashFlow,score_margin_grossProfit,score_margin_netIncomeFromOperations,total_score,rank_quantl
0,ZTS,1.0,1.0,1.0,1.0,1.0,1.0,0,0,1,1,8.0,0.048708
1,ODFL,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.048708
2,ORLY,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.048708
3,CNC,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.048708
4,OTIS,1.0,1.0,1.0,1.0,1.0,1.0,1,0,1,0,8.0,0.048708
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,F,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987078
499,APTV,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987078
500,LUV,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987078
501,GL,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.987078


### Cache dataset. 

In [13]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_annual_rankbyscore.csv") 
# df_rank_fundamental.to_csv(filepath, index=False) 

### Rank by within sector performance difference. 

In [14]:
# Rank the performance difference within sector. 
usecols = ["ticker", "fiscalDateEnding", "gics_sector"] + list(FINANCIAL_EVAL_QUALITY.keys()) 
df_within_sector_diff = within_sector_diff(df_finstate_ann_ext, df_snp_info, usecols, yearspec=("fiscalDateEnding", 2018), groupcols=["gics_sector"]) 
df_rank_within_sector_diff = rank_within_sector_diff(df_within_sector_diff, groupcols=["gics_sector"]) 

# Preview. 
df_rank_within_sector_diff 

Ranking the performance difference within sector.


,fiscalDateEnding,gics_sector,ticker,growth_totalRevenue,growth_grossProfit,growth_netIncomeFromOperations,growth_ebit,growth_netIncome_x,growth_operatingCashflow,turnover_asset,turnover_operatingCashFlow,margin_grossProfit,margin_netIncomeFromOperations,rank_quantl
0,2018-12-31,Financials,CINF,-0.117534,0.051876,-0.462035,-0.726954,-0.785009,0.260820,0.043694,-0.135557,-0.789631,-0.118758,0.061538
1,2018-06-30,Health Care,CTLT,0.061259,-0.014604,-1.703641,0.106777,-1.397651,-1.396041,-0.457360,-0.001906,-0.192114,-0.037670,0.266667
2,2018-09-30,Industrials,ROK,-0.053702,-0.780147,-0.805470,0.003327,-0.742821,0.028178,0.129212,0.036425,0.027601,-0.022455,0.492958
3,2018-12-31,Information Technology,FTNT,0.066316,0.044888,7.583929,-37.830544,8.866559,-0.185764,-0.105674,0.091921,0.231618,0.022148,0.863014
4,2018-12-31,Consumer Discretionary,CZR,0.163030,3.884938,0.032299,4.580399,0.414581,2.067112,-0.896015,0.019573,0.219308,-0.055161,0.657895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,2018-06-30,Information Technology,BR,-0.086525,-0.050188,-1.686262,-38.840241,-0.489682,0.083817,0.622695,-0.104302,-0.255307,-0.064354,0.205479
487,2018-12-31,Real Estate,PLD,0.025317,-0.716697,-0.233260,-0.148157,-0.253771,-0.053829,-0.152435,0.175255,-0.054858,0.386476,0.250000
488,2018-12-31,Communication Services,PARA,-0.093009,-0.192925,-0.137579,-2.982744,1.289559,0.106924,-0.180966,-0.004535,-0.324683,0.117250,0.217391
489,2018-12-31,Consumer Discretionary,PHM,0.073321,15.147599,-7.964393,0.570438,1.405226,1.762331,-0.213086,-0.009934,0.459822,0.634440,0.877193


### Cache dataset. 

In [15]:
# # Uncomment this part to cache the dataset. 
# filepath = os.path.join(DIR_DATASET_FUNDAMENTAL, "financial_statement_annual_rankbywithinsector.csv") 
# df_rank_within_sector_diff.to_csv(filepath, index=False) 

## Output top N ticker ranks. 

### By Score. 

In [16]:
ticker_rank_fundamental = df_rank_fundamental \
	.loc[df_rank_fundamental["rank_quantl"] <= 0.05, "ticker"] \
	.unique() 

ticker_rank_fundamental = set(ticker_rank_fundamental) 
ticker_rank_fundamental 

{'AAPL',
 'ADBE',
 'ALGN',
 'AMAT',
 'AMZN',
 'ANET',
 'ARE',
 'CBRE',
 'CDNS',
 'CE',
 'CMG',
 'CNC',
 'CPRT',
 'DG',
 'DHI',
 'DPZ',
 'EPAM',
 'EW',
 'GOOG',
 'GOOGL',
 'GRMN',
 'ICE',
 'IDXX',
 'INTU',
 'IP',
 'ISRG',
 'KLAC',
 'LRCX',
 'MA',
 'META',
 'MSFT',
 'MU',
 'NVDA',
 'ODFL',
 'ORLY',
 'OTIS',
 'PFE',
 'POOL',
 'REGN',
 'SIVB',
 'TER',
 'TFC',
 'TROW',
 'TSCO',
 'ULTA',
 'UPS',
 'VRTX',
 'ZTS'}

### By within sector performance difference. 

In [17]:
ticker_within_sector_diff = df_rank_within_sector_diff \
	.loc[df_rank_within_sector_diff["rank_quantl"] <= 0.05, "ticker"] \
	.unique() 

ticker_within_sector_diff = set(ticker_within_sector_diff) 
ticker_within_sector_diff 

{'AAL',
 'AFL',
 'AIG',
 'ALK',
 'BKR',
 'CSCO',
 'CVS',
 'FIS',
 'KHC',
 'MAA',
 'NEM',
 'NLSN',
 'NRG',
 'NWL',
 'QCOM',
 'RE',
 'WHR',
 'XRAY',
 'ZBH'}

### Top N tickers on various evaluation methods. 

In [18]:
# Get top ranking tickers via various fundamental evaluation methods. 
ticker_rank_fundamental.intersection(ticker_within_sector_diff) 

set()